In [2]:
import numpy as np, matplotlib.pyplot as plt, os, glob
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

## The Three-band GPR method using SVR based on Blix et al (442.5, 673.5, 681.25)

In [3]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

training = pd.read_csv(body, usecols=(2,8,9,16))
training.head()


,442.5,673.75,681.25,Chl
0,0.00470,0.000161,0.000170,0.1
1,0.00590,0.000401,0.000396,0.1
2,0.00657,0.000642,0.000621,0.1
3,0.00699,0.000883,0.000848,0.1
4,0.00726,0.001120,0.001070,0.1


In [4]:
olci_bands = training[['442.5','673.75','681.25']]
olci_bands

,442.5,673.75,681.25
0,0.00470,0.000161,0.000170
1,0.00590,0.000401,0.000396
2,0.00657,0.000642,0.000621
3,0.00699,0.000883,0.000848
4,0.00726,0.001120,0.001070
...,...,...,...
706334,0.00443,0.013000,0.014600
706335,0.00437,0.013000,0.014600
706336,0.00431,0.013000,0.014600
706337,0.00425,0.013000,0.014600


In [5]:
chl_a = training['Chl']
chl_a

0          0.1
1          0.1
2          0.1
3          0.1
4          0.1
          ... 
706334    28.0
706335    28.0
706336    28.0
706337    28.0
706338    28.0
Name: Chl, Length: 706339, dtype: float64

In [6]:
#convert the above df and series to numpy arrays

X = olci_bands.to_numpy()
Y = chl_a.to_numpy()

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.10, random_state = 100)
print(x_train, x_train.shape)
print(y_train, y_train.shape)


[[0.00892  0.0142   0.0145  ]
 [0.000521 0.00142  0.00168 ]
 [0.00619  0.00607  0.0058  ]
 ...
 [0.00299  0.00498  0.00502 ]
 [0.00395  0.00689  0.00654 ]
 [0.00141  0.00404  0.00487 ]] (70633, 3)
[ 8.9   6.5   0.7  ... 17.    7.25 24.5 ] (70633,)


In [26]:
# use a SVR algorithm to fit model
model = SVR(C=50000, epsilon=10)
model.fit(x_train, y_train)

print("Model R squared is {} ".format(model.score(x_train, y_train)))

Model R squared is 0.44974716479023324 


In [9]:

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

validation = pd.read_csv(body,usecols=(2,8,9,16))
validation.head()


,442.5,673.75,681.25,Chl
0,0.005157,0.014518,0.014777,33.599998
1,0.008432,0.013225,0.013581,27.900000
2,0.006706,0.010274,0.010327,15.400000
3,0.007841,0.014313,0.014693,20.299999
4,0.009737,0.006933,0.007043,6.800000


In [10]:
olci_bands_valid = validation[['442.5','673.75','681.25']]
chl_a_valid = validation['Chl']
x_valid = olci_bands_valid.to_numpy()
y_valid = chl_a_valid.to_numpy()

In [29]:
print(x_valid)
print(y_valid)

[[0.0051567 0.014518  0.014777 ]
 [0.0084318 0.013225  0.013581 ]
 [0.0067062 0.010274  0.010327 ]
 [0.0078414 0.014313  0.014693 ]
 [0.0097373 0.0069333 0.0070428]
 [0.0032425 0.0031065 0.0033998]
 [0.0053608 0.0033237 0.003534 ]
 [0.0041226 0.0043885 0.0045799]
 [0.0052102 0.0053961 0.0054387]
 [0.029003  0.025103  0.024394 ]
 [0.0041364 0.0016112 0.0017744]
 [0.015279  0.0053678 0.0051921]
 [0.0040998 0.0041785 0.0049173]
 [0.0079203 0.010951  0.011145 ]
 [0.0030886 0.0079583 0.0084067]
 [0.0082079 0.011346  0.011484 ]
 [0.0049242 0.0048877 0.0049536]
 [0.0045852 0.0048752 0.0050463]
 [0.0052947 0.0064628 0.0064235]
 [0.0056408 0.0042591 0.0043102]
 [0.010178  0.011939  0.011979 ]
 [0.0088781 0.0099788 0.0099191]
 [0.0076032 0.0062852 0.0062816]
 [0.021322  0.012138  0.011735 ]
 [0.0065809 0.0073752 0.0074224]
 [0.0078011 0.0036105 0.0036541]
 [0.0056952 0.0017497 0.0017444]
 [0.0056035 0.0014091 0.0015119]
 [0.016216  0.0040376 0.0039326]
 [0.015491  0.0041947 0.0040752]
 [0.019663

In [27]:
# predict using fitted model 
y_pred = model.predict(x_valid)

# calculate NRMSE
from sklearn.metrics import mean_squared_error, make_scorer
RMSE = mean_squared_error(y_valid, y_pred, squared=False)
N = y_valid.max() - y_valid.min()
NRMSE = RMSE/N

print("Normalized Root Mean Squared Error is {}".format(NRMSE))

Normalized Root Mean Squared Error is 0.1690209424502465


In [ ]:
# using cross validation with 10 splits
#create scoring function NRMSE for CV
def nrmse(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    norm = y_true.max() - y_true.min()
    return rmse/norm
score = make_scorer(nrmse, greater_is_better=False)
cross_val_nrmse = cross_val_score(model, x_train, y_train, cv=10, scoring=score)
print(cross_val_nrmse, cross_val_nrmse.mean())

## The Three-band GPR method using Random Forest based on Blix et al (442.5, 673.5, 681.25)

In [35]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.010, random_state = 123)
print(x_train, x_train.shape)
print(y_train, y_train.shape)

[[0.00532 0.0113  0.0114 ]
 [0.00364 0.0132  0.0139 ]
 [0.00288 0.00618 0.00725]
 ...
 [0.00268 0.0025  0.00323]
 [0.00159 0.00257 0.00333]
 [0.0101  0.0116  0.0118 ]] (7063, 3)
[36.7 24.  94.8 ...  6.   9.  29.2] (7063,)


In [36]:
# using the Random Forest algorithm
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 200, max_depth=None, min_samples_split = 2, min_samples_leaf= 1, max_leaf_nodes=None, bootstrap=True)
model.fit(x_train, y_train)
print("Model R squared is {} ".format(model.score(x_train, y_train)))

Model R squared is 0.9448588106341957 


In [24]:
# predict using RFR fitted model 
y_pred = model.predict(x_valid)

# calculate NRMSE
from sklearn.metrics import mean_squared_error, make_scorer
RMSE = mean_squared_error(y_valid, y_pred, squared=False)
N = y_valid.max() - y_valid.min()
NRMSE = RMSE/N

print("Normalized Root Mean Squared Error is {}".format(NRMSE))

Normalized Root Mean Squared Error is 0.18501396886872665


In [25]:
# using cross validation with 10 splits
#create scoring function NRMSE for CV
def nrmse(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    norm = y_true.max() - y_true.min()
    return rmse/norm
score = make_scorer(nrmse, greater_is_better=False)
cross_val_nrmse = cross_val_score(model, x_train, y_train, cv=10, scoring=score)
print(cross_val_nrmse, cross_val_nrmse.mean())

[-0.10369786 -0.09987849 -0.0996529  -0.09997678 -0.1012879  -0.10065611
 -0.09950694 -0.09938346 -0.09869365 -0.10068435] -0.10034184392264604


In [38]:
### Insufficient resources to run cell
## find optimum/best parameters using GridSearch
RFR = RandomForestRegressor()
parameters = {'n_estimators': [50, 100, 200, 500], 'max_depth': [None, 50, 200, 500], 'min_samples_split': [2, 10, 100], 'min_samples_leaf':[1, 50, 100], \
             'max_leaf_nodes': [None, 10, 100, 1000], 'bootstrap': [True, False]}
     
GSC = GridSearchCV(RFR, parameters)
GSC.fit(x_train, y_train)
 
print(GSC.get_params())
#print(sorted(GSC.cv_results_.keys()))
print(GSC.best_estimator_)
print(GSC.best_params_)
print(GSC.best_score_)

KeyboardInterrupt: 